# BASELINE MODEL
-----------------
-----------------
-----------------

In [1]:
import pandas as pd
import numpy as np
import time
import gc
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, GlobalMaxPooling2D, LSTM, MaxPooling2D, Flatten
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.svm import SVC

In [7]:
!free -h

               total        used        free      shared  buff/cache   available
Mem:            15Gi       7.5Gi       3.1Gi       247Mi       4.9Gi       7.5Gi
Swap:          976Mi       0.0Ki       976Mi


In [3]:
# This df will be a row-concatenated version of the same channels
# baseline = pd.read_csv('../../baseline.csv', delimiter=',')
# baseline.drop('Unnamed: 0', inplace=True, axis=1)

In [3]:
# del df
gc.collect()

62

In [6]:
df = pd.read_csv('../data/dfTensor2.csv', delimiter=';', encoding='latin-1')
df.drop('Unnamed: 0', inplace=True, axis=1)

In [8]:
df['F8'].iloc[0]

'[3989.23077, 3983.589744, 3987.692308, 3991.794872, 3992.307693, 3990.256411, 3993.333334, 4000.0, 3994.871795, 3985.128206, 3995.897436, 4001.538461, 3986.153847, 3982.051283, 3990.256411, 3994.358975, 3981.538462, 3961.025642, 3968.205129, 3989.23077, 3980.512821, 3973.333334, 3997.948718, 4009.230769, 3998.974359, 3998.461539, 3999.48718, 3997.435898, 3999.48718, 4001.025641, 3994.871795, 3987.179488, 3987.692308, 3992.307693, 3987.692308, 3980.0, 3984.102565, 3994.358975, 3991.794872, 3982.564103, 3980.0, 3978.461539, 3978.461539, 3989.23077, 3998.461539, 3998.461539, 3996.410257, 3978.974359, 3966.153847, 3978.974359, 3987.692308, 3985.128206, 3993.846154, 3995.384616, 3974.871795, 3957.948718, 3965.128206, 3990.769231, 4007.179487, 3998.974359, 3981.025642, 3970.256411, 3976.923077, 3993.333334, 3992.820513, 3978.461539, 3976.923077, 3975.897436, 3973.846154, 3984.615385, 3986.153847, 3978.974359, 3980.0, 3982.051283, 3981.538462, 3983.589744, 3978.974359, 3971.282052, 3972.8205

In [13]:
# df['O1'] = df['O1'].apply(eval)
# df['O1']

In [3]:
# loading dataframes for each channel, pick and choose at will

# occ0 = pd.read_csv('../data/occ0Exp.csv', delimiter=',')
# occ1 = pd.read_csv('../data/occ1Exp.csv', delimiter=',')
# fefF3 = pd.read_csv('../data/fefF3Exp.csv', delimiter=',')
# fefF4 = pd.read_csv('../data/fefF4Exp.csv', delimiter=',')
# fefF7 = pd.read_csv('../data/fefF7Exp.csv', delimiter=',')
# fefF8 = pd.read_csv('../data/fefF8Exp.csv', delimiter=',')
# temT7 = pd.read_csv('../data/temT7Exp.csv', delimiter=',')
# temT8 = pd.read_csv('../data/temT8Exp.csv', delimiter=',')
# pfcAF3 = pd.read_csv('../data/pfcAF3Exp.csv', delimiter=',')
# pfcAF4 = pd.read_csv('../data/pfcAF4Exp.csv', delimiter=',')
# motFC5 = pd.read_csv('../data/motFC5Exp.csv', delimiter=',')
# motFC6 = pd.read_csv('../data/motFC6Exp.csv', delimiter=',')
# parP7 = pd.read_csv('../data/parP7Exp.csv', delimiter=',')
# parP8 = pd.read_csv('../data/parP8Exp.csv', delimiter=',')

# loading same datasets expanded with filter processing

# occ0 = pd.read_csv('../data/occ0Proc.csv', delimiter=',')
# occ1 = pd.read_csv('../data/occ1Proc.csv', delimiter=',') 
# fefF3 = pd.read_csv('../data/fefF3Proc.csv', delimiter=',')
# fefF4 = pd.read_csv('../data/fefF4Proc.csv', delimiter=',')
# fefF7 = pd.read_csv('../data/fefF7Proc.csv', delimiter=',')
# fefF8 = pd.read_csv('../data/fefF8Proc.csv', delimiter=',') 
# temT7 = pd.read_csv('../data/temT7Proc.csv', delimiter=',')
# temT8 = pd.read_csv('../data/temT8Proc.csv', delimiter=',')
# pfcAF3 = pd.read_csv('../data/pfcAF3Proc.csv', delimiter=',')
# pfcAF4 = pd.read_csv('../data/pfcAF4Proc.csv', delimiter=',')
# motFC5 = pd.read_csv('../data/motFC5Proc.csv', delimiter=',')
# motFC6 = pd.read_csv('../data/motFC6Proc.csv', delimiter=',')
# parP7 = pd.read_csv('../data/parP7Proc.csv', delimiter=',')
# parP8 = pd.read_csv('../data/parP8Proc.csv', delimiter=',')

# occ0 = pd.read_csv('../data/occ0Wide.csv', delimiter=',')
# occ1 = pd.read_csv('../data/occ1Wide.csv', delimiter=',') 
# fefF3 = pd.read_csv('../data/fefF3Wide.csv', delimiter=',')
# fefF4 = pd.read_csv('../data/fefF4Wide.csv', delimiter=',')
# fefF7 = pd.read_csv('../data/fefF7Wide.csv', delimiter=',')
# fefF8 = pd.read_csv('../data/fefF8Wide.csv', delimiter=',') 
# temT7 = pd.read_csv('../data/temT7Wide.csv', delimiter=',')
# temT8 = pd.read_csv('../data/temT8Wide.csv', delimiter=',')
# pfcAF3 = pd.read_csv('../data/pfcAF3Wide.csv', delimiter=',')
# pfcAF4 = pd.read_csv('../data/pfcAF4Wide.csv', delimiter=',')
# motFC5 = pd.read_csv('../data/motFC5Wide.csv', delimiter=',')
# motFC6 = pd.read_csv('../data/motFC6Wide.csv', delimiter=',')
# parP7 = pd.read_csv('../data/parP7Wide.csv', delimiter=',')
# parP8 = pd.read_csv('../data/parP8Wide.csv', delimiter=',')

In [ ]:
# del occ0
# del occ1
# del fefF3
# del fefF4
# del fefF7
# del fefF8
# del temT7
# del temT8
# del pfcAF3
# del pfcAF4
# del motFC5
# del motFC6
# del parP7
# del parP8

# gc.collect()

In [5]:
# loading only the dataframes from channels positively correlated with code, to save RAM
# corrcols = [occ1, fefF4, fefF8, pfcAF4, motFC6, parP8] #removed motFC5 for being a diff shape

Correlations of averaged channels, if there is a need to pick and choose to save RAM:

<sub>
code          1.000000
    
&nbsp;
    
occ0Data     -0.000385

&nbsp;
    
occ1Data      0.001626
    
&nbsp;

fefF3Data    -0.005483

&nbsp;
    
fefF4Data     0.008288

&nbsp;
    
fefF7Data    -0.004799

&nbsp;
        
fefF8Data     0.006907

&nbsp;    
    
temT7Data    -0.002657

&nbsp;    
    
temT8Data    -0.001037

&nbsp;    
    
pfcAF3Data   -0.007512

&nbsp;    
    
pfcAF4Data    0.007591

&nbsp;    
    
motFC5Data    0.003757

&nbsp;    
    
motFC6Data    0.001372

&nbsp;    
    
parP7Data    -0.003000

&nbsp;    
    
parP8Data     0.002366
</sub>

In [4]:
!free -h

               total        used        free      shared  buff/cache   available
Mem:            15Gi       7.7Gi       946Mi       562Mi       6.9Gi       6.9Gi
Swap:          976Mi        11Mi       965Mi


In [7]:
# If individual columns are loaded, use this cell to process them and aggregate group them by event
# This gets rid of the time dimension though and will make the model be highly inaccurate

# dfs = [occ0, occ1, fefF3, fefF4, fefF7, fefF8, 
#        temT7, temT8, pfcAF3, pfcAF4, motFC5, motFC6, parP7, parP8]
# for i in dfs: #dfs:
# #     i.drop('Unnamed: 0.1', inplace=True, axis=1)
#     i.drop(['Unnamed: 0', 'size'], inplace=True, axis=1)
#     i['data'].astype(float, copy=False)
#     print(i.columns)
# #     i = i.groupby('event').mean()
# #     print(i.columns)

In [10]:
# non loop version of above's last line
# occ1, fefF4, fefF8, pfcAF4, motFC5, motFC6, parP8 for corrcols

# occ0 = occ0.groupby('event').mean()
# occ1 = occ1.groupby('event').mean()
# fefF3 = fefF3.groupby('event').mean()
# fefF4 = fefF4.groupby('event').mean()
# fefF7 = fefF7.groupby('event').mean()
# fefF8 = fefF8.groupby('event').mean()
# temT7 = temT7.groupby('event').mean()
# temT8 = temT8.groupby('event').mean()
# pfcAF3 = pfcAF3.groupby('event').mean()
# pfcAF4 = pfcAF4.groupby('event').mean()
# motFC5 = motFC5.groupby('event').mean() #this one has less rows than the rest, why?
# motFC6 = motFC6.groupby('event').mean()
# parP7 = parP7.groupby('event').mean()
# parP8 = parP8.groupby('event').mean()


In [8]:
## Check each channel dataframe for null vals 
# for i in corrcols:
#     print(max(i.isna().sum()))

In [58]:
## Renaming data column to specify channel name in individual channel dataframes, so when they are concatenated, the full df doesn't have multiple cols with the same names
## (use  occ1, fefF4, fefF8, pfcAF4, motFC6, parP8 for corrcols)

# occ0.rename(columns={'data': 'occ0data'}, inplace=True)
# occ1.rename(columns={'data':'occ1data', 'ICAdata':'occ1ICAdata', 'SGFdata':'occ1SGFdata'}, inplace=True)
# fefF3.rename(columns={'data': 'fefF3data'}, inplace=True)
# fefF4.rename(columns={'data':'fefF4data', 'ICAdata':'fefF4ICAdata', 'SGFdata':'fefF4SGFdata'}, inplace=True)
# fefF7.rename(columns={'data': 'fefF7data'}, inplace=True)
# fefF8.rename(columns={'data':'fefF8data', 'ICAdata':'fefF8ICAdata', 'SGFdata':'fefF8SGFdata'}, inplace=True)
# temT7.rename(columns={'data': 'temT7data'}, inplace=True)
# temT8.rename(columns={'data': 'temT8data'}, inplace=True)
# pfcAF3.rename(columns={'data': 'pfcAF3data'}, inplace=True)
# pfcAF4.rename(columns={'data':'pfcAF4data', 'ICAdata':'pfcAF4ICAdata', 'SGFdata':'pfcAF4SGFdata'}, inplace=True)
# motFC5.rename(columns={'data': 'motFC5data'}, inplace=True)
# motFC6.rename(columns={'data':'motFC6data', 'ICAdata':'motFC6ICAdata', 'SGFdata':'motFC6SGFdata'}, inplace=True)
# parP7.rename(columns={'data': 'parP7data'}, inplace=True)
# parP8.rename(columns={'data':'parP8data', 'ICAdata':'parP8ICAdata', 'SGFdata':'parP8SGFdata'}, inplace=True)


In [14]:
# occ0.columns, occ1.columns, fefF7.columns, fefF8.columns
# # dfs[0] #this wasn't changed with the above functions, it's only used for calling the original datasets as they were read in
# dfsProcessed = [occ0, occ1, fefF3, fefF4, fefF7, fefF8, 
#        temT7, temT8, pfcAF3, pfcAF4, motFC5, motFC6, parP7, parP8]

## Remaking this list included processed dataframes
# corrcolsProcessed = [occ1, fefF4, fefF8, pfcAF4, motFC6, parP8] #new vars because the original list has the unprocessed dataframes in memory it seems

In [15]:
## If using individual channel dataframes, use this cell to turn them into one df
# full = pd.concat(corrcolsProcessed, axis=1, join='inner')

In [18]:
# full.columns.duplicated()
# # Since the concat created duplicate code columns, using this trick to delete them:
# #https://stackoverflow.com/questions/14984119/python-pandas-remove-duplicate-columns
# #Although a different type of join might prevent the need for this
# full = full.loc[:,~full.columns.duplicated()].copy()
# #full = full.drop(['size', 'id' ], axis=1) # If any extraneous cols are leftover
# full.dtypes

In [9]:
# full

In [10]:
#corrMatFull = full.corr()
#corrMatFull['code']
# check = pd.read_csv('../data/occ1Exp.csv', delimiter=',')
# print(check.corr()) # code correlates to data in this channel by 0.001562 
# del check 

# check = pd.read_csv('../data/fefF7Exp.csv', delimiter=',')
# print(check.corr()) # code correlates to data in this channel by -0.004390.
# del check 

# check = pd.read_csv('../data/occ0Exp.csv', delimiter=',')
# print(check.corr()) # code correlates to data in this channel by -0.000308 
# del check 

# check = pd.read_csv('../data/fefF8Exp.csv', delimiter=',')
# print(check.corr()) # code correlates to data in this channel by 0.006184
# del check 

In [20]:
# full.plot(y=['occ0Data', 'occ1Data', 'fefF7Data', 'fefF8Data'], alpha=.7 ); 

In [85]:
!free -h

               total        used        free      shared  buff/cache   available
Mem:            15Gi        13Gi       334Mi       227Mi       1.7Gi       1.4Gi
Swap:          976Mi        45Mi       931Mi


--------------
## Baseline Model: Only O1, O2, F7, F8; No Preprocessing
## Baseline+ : Basline+Preprocessing

--------------

&nbsp;


In [17]:
df.iloc[0][0]

'[4203.076923, 4193.333333, 4194.871794, 4207.179487, 4220.51282, 4222.564102, 4217.435897, 4212.307692, 4210.25641, 4214.358974, 4216.923076, 4210.76923, 4210.25641, 4215.384615, 4216.410256, 4214.358974, 4212.307692, 4211.282051, 4212.820512, 4215.897435, 4214.358974, 4213.846153, 4224.102564, 4231.282051, 4230.25641, 4229.743589, 4222.051282, 4214.358974, 4218.974358, 4222.564102, 4218.974358, 4223.589743, 4230.25641, 4230.76923, 4227.692307, 4224.102564, 4227.179487, 4232.307692, 4230.76923, 4229.743589, 4233.333333, 4233.333333, 4232.820512, 4236.410256, 4234.871794, 4232.307692, 4238.974358, 4249.230769, 4251.794871, 4249.230769, 4246.153846, 4244.615384, 4246.153846, 4246.153846, 4247.179487, 4257.948717, 4267.692307, 4268.205128, 4265.641025, 4266.153846, 4264.102564, 4261.538461, 4267.692307, 4271.282051, 4265.128205, 4267.692307, 4282.051282, 4285.128205, 4280.0, 4287.692307, 4296.410256, 4289.743589, 4279.487179, 4277.435897, 4279.487179, 4286.153846, 4295.384615, 4297.43589

In [183]:
## Setting up X and y, train-test split for "full"
# X = full.drop('code', axis=1)
# XExperiment = full.drop(['code', 'occ1data', 'fefF4data',
#                          'fefF8data', 'pfcAF4data', 'motFC6data', 'parP8data',
#                          'occ1SGFdata', 'fefF4SGFdata','fefF8SGFdata', 'pfcAF4SGFdata', #TODO change to pfc
#                          'motFC6SGFdata', 'parP8SGFdata'], axis=1) 
# y = full['code']
# print(X.columns, X.iloc[0], y.iloc[0], X.shape, y.shape)
# XTrain, XTest, yTrain, yTest = train_test_split(XExperiment, y)

In [9]:
# Setting up X and y
X = df.drop('code', axis=1)
y = df['code']

In [10]:
#Since the csv is formatted wrong, something with read_csv is encoding the lists as strings, run this to turn them back to lists
X = X.applymap(eval) # takes a while
print(X.iloc[0], type(y.iloc[0]), X.shape, y.shape)

O1     [4203.076923, 4193.333333, 4194.871794, 4207.1...
O2     [4229.743589, 4216.923076, 4207.179487, 4215.3...
F3     [4538.461538, 4528.717948, 4524.615384, 4526.1...
F4     [4682.051282, 4667.179487, 4662.051282, 4669.2...
F7     [4489.230769, 4475.384615, 4474.358974, 4486.6...
F8     [3989.23077, 3983.589744, 3987.692308, 3991.79...
T7     [4497.948717, 4498.461538, 4494.871794, 4497.9...
T8     [4506.666666, 4501.025641, 4496.923076, 4496.9...
AF3    [4395.384615, 4382.564102, 4377.435897, 4387.1...
AF4    [4078.461538, 4062.564102, 4055.897435, 4065.6...
FC5    [4207.692307, 4205.641025, 4200.51282, 4194.35...
FC6    [4227.692307, 4215.384615, 4210.76923, 4221.02...
P7     [4203.076923, 4192.820512, 4194.871794, 4204.6...
P8     [4245.641025, 4236.410256, 4218.461538, 4220.0...
Name: 0, dtype: object <class 'numpy.int64'> (65034, 14) (65034,)


In [12]:
# train-test split for "df"
XTrain, XTest, yTrain, yTest = train_test_split(X, y)

In [13]:
type(X['T7'].iloc[0][0])

float

In [14]:
!cowsay moooooo... braaains...

 ________________________
< moooooo... braaains... >
 ------------------------
        \   ^__^
         \  (oo)\_______
            (__)\       )\/\
                ||----w |
                ||     ||


In [29]:
XTrain.shape, XTest.shape, yTrain.shape, yTest.shape

((48775, 14), (16259, 14), (48775,), (16259,))

In [30]:
del df
gc.collect()

NameError: name 'df' is not defined

In [31]:
XTrainNP = XTrain.applymap(np.array)
# type(XTrainNP['T7'][0])
XTrainNP = XTrainNP.to_numpy()
XTrainNP.shape

(48775, 14)

In [32]:
!free -h

               total        used        free      shared  buff/cache   available
Mem:            15Gi        10Gi       555Mi       286Mi       4.5Gi       4.4Gi
Swap:          976Mi        25Mi       951Mi


In [19]:
XTrainNP.flatten().shape

(682850,)

In [20]:
48775*14

682850

In [21]:
# XTrainNP = tf.reshape(np.array(XTrainNP), (48775, 256, 1, 14))
XTrainNP = np.reshape(XTrainNP, newshape=(256, 1, 14))
# ohe = OneHotEncoder(sparse=False)
# # to fix error, try this: https://www.youtube.com/watch?v=k_VAKyzggJI
# from sklearn.preprocessing import LabelEncoder
# lb = LabelEncoder()
# yTrainOHE = lb.fit_transform(yTrain)
# yTrainOHE = ohe.fit_transform(yTrain.to_numpy().reshape(-1,1))

# ohe0 = OneHotEncoder(sparse=False)
# yTestOHE = ohe0.fit(yTrain.to_numpy().reshape(-1,1)) # Daniel said to do this but I can't remember why

ValueError: cannot reshape array of size 682850 into shape (256,1,14)

In [55]:
del model
gc.collect()

14876

In [22]:
model = Sequential()

In [23]:
model.add(Conv2D(filters=64, kernel_size=1, activation='relu', input_shape=(256,14,1)))
model.add(MaxPooling2D(pool_size=(1,1)))
model.add(Conv2D(filters=64, kernel_size=1, activation='relu'))
model.add(MaxPooling2D(pool_size=(1,1)))
model.add(Conv2D(filters=64, kernel_size=1, activation='relu'))
model.add(MaxPooling2D(pool_size=(1,1)))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
# model.add(Dense(32, activation='relu'))
# model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(11, activation='softmax'))

# model.add(LSTM(7, input_shape=(1,14,1), activation='tanh', 
#                recurrent_activation='sigmoid'))

In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 14, 64)       128       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 256, 14, 64)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 256, 14, 64)       4160      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 256, 14, 64)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 256, 14, 64)       4160      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 256, 14, 64)      0

In [25]:
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
results = model.fit(XTrain, yTrainOHE, batch_size=20,epochs=30, verbose=1)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type list).

Best accuracy for the unprocessed signal with all channels was about .1020

In 6/14 channels, adding SGF data was about the same

In 6/14 channels, adding SGF and dropping raw data resulted in really low accuracy and NaN loss

In 6/14 channels, ICA only had a accuracy of about .1000

6/14 channels, unproccessed+SGF+ICA, 22,059 params: NaN loss again, what's wrong with the data? woe be the futility of mine ways

6/14 channels, SGF only, 9,771 params, accuracy still around .1000

6/14 channels, raw, 9,771 params, accuracy about .1000, so the .0020 increase was only from the extra channels. Processing doesn't seem to change the accuracy at all.

----------
Using vector data rows ("wide"), I get this error:
```
UnimplementedError                        Traceback (most recent call last)
<ipython-input-59-f6344dc87bbc> in <module>
      1 model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
----> 2 results = model.fit(XTrain, yTrainOHE, batch_size=20,epochs=30, verbose=1)

...

Cast string to float is not supported
	 [[{{node sequential_4/Cast}}]] [Op:__inference_train_function_2630]
```

but the prep function seems to work fine: column is series, col[0] is list, and col[0][0] is float
```
(learn-env) :~/flatiron/cap/capstoneProject/workingNotebooks$ python dataPrepV2.py 
<class 'pandas.core.series.Series'> <class 'list'> <class 'float'>
```

-[] TODO: OHE test, validation set

### Support Vector Machine Attempt

In [34]:
svc = SVC()
svc.fit(X, y)

ValueError: setting an array element with a sequence.

In [116]:
svc.score(XTest, yTest)

0.10234331754720463

A quick support vector classifier ended up getting the same accuracy, 0.10234331754720463

### Echo State Network Attempt

In [190]:
import reservoirpy as rpy
from reservoirpy.nodes import Reservoir, Ridge, FORCE, ESN

readout = Ridge(ridge=1e-7)
train_states = reservoir.run(XTrain.to_numpy().reshape(-1, 1), reset=True)
# train_statesy = reservoir.run(yTrain.to_numpy().reshape(-1, 1), reset=True)
readout = readout.fit(train_states, yTrain.to_numpy().reshape(-1, 1), warmup=10)
test_states = reservoir.run(XTest.to_numpy().reshape(-1, 1))
yPred = readout.run(test_states)

reservoir = Reservoir(100, lr=0.5, sr=0.9)
ridge = Ridge(ridge=1e-7)

esn_model = reservoir >> ridge

esn_model = esn_model.fit(XTrain.to_numpy().reshape(-1, 1), yTrain.to_numpy().reshape(-1, 1).to_numpy().reshape(-1, 1), warmup=10)
print(reservoir.is_initialized, readout.is_initialized, readout.fitted)

yPred = esn_model.run(XTest.to_numpy().reshape(-1, 1))


ValueError: cannot reshape array of size 292650 into shape (48775,1)

While it was nice to learn how to set one of these up, I realized after a lot of tuning the reservoirpy library doesn't seem to have good interop with pandas. Might have interesting results with 1D timestamp and 1D value arrays, but not a multidimensional dataframe

In [209]:
readout1 = Ridge(ridge=1e-7)
train_states1 = reservoir.run(full['pfcAF4SGFdata'].to_numpy().reshape(-1, 1), reset=True)
# train_statesy = reservoir.run(yTrain.to_numpy().reshape(-1, 1), reset=True)
readout1 = readout1.fit(train_states1, full['code'].to_numpy().reshape(-1, 1), warmup=10)
test_states1 = reservoir.run((full['pfcAF4SGFdata'].to_numpy().reshape(-1, 1)))
# yPred1 = readout.run(test_states1)

reservoir1 = Reservoir(100, lr=0.5, sr=0.9)
ridge1 = Ridge(ridge=1e-7)

esn_model1 = reservoir >> ridge

esn_model1 = esn_model1.fit(full['pfcAF4SGFdata'].to_numpy().reshape(-1, 1), 
                             full['code'].to_numpy().reshape(-1, 1) ,warmup=10)
print(reservoir.is_initialized, readout.is_initialized, readout.fitted)

yPred1 = esn_model.run((full['pfcAF4SGFdata'].to_numpy().reshape(-1, 1)))

Running Model-7:   0%|          | 0/1 [00:00<?, ?it/s]
Running Model-7: 608it [00:00, 6077.69it/s]           
Running Model-7: 1184it [00:00, 5977.77it/s]
Running Model-7: 1774it [00:00, 5953.20it/s]
Running Model-7: 2340it [00:00, 5861.04it/s]
Running Model-7: 2958it [00:00, 5950.73it/s]
Running Model-7: 3509it [00:00, 5809.06it/s]
Running Model-7: 4114it [00:00, 5877.67it/s]
Running Model-7: 4698it [00:00, 5864.64it/s]
Running Model-7: 5266it [00:00, 5805.29it/s]
Running Model-7: 5856it [00:01, 5832.72it/s]
Running Model-7: 6438it [00:01, 5827.26it/s]
Running Model-7: 7041it [00:01, 5886.27it/s]
Running Model-7: 7640it [00:01, 5915.91it/s]
Running Model-7: 8228it [00:01, 5904.99it/s]
Running Model-7: 8820it [00:01, 5909.08it/s]
Running Model-7: 9416it [00:01, 5922.98it/s]
Running Model-7: 10023it [00:01, 5964.50it/s]
Running Model-7: 10635it [00:01, 6009.18it/s]
Running Model-7: 11236it [00:01, 5912.81it/s]
Running Model-7: 11828it [00:02, 5818.73it/s]
Running Model-7: 12411it [00:02

Fitting node Ridge-19...


Running Model-6: 1195it [00:00, 5902.90it/s]          

True False False


Running Model-6: 65034it [00:10, 6031.98it/s]


In [220]:
set(yPred1.flatten()), (mean())

{4.494217014863551}

In [ ]:
# TODO use exploded data of one event from original CSV, predict brain activity with ESN:
# forecast, generate artificial brainwaves of looking at a digit

# TODO explore possibility of using matplotlib plots of signals as training data
# like in https://arxiv.org/abs/1511.06448

# in the future, try iterating ESNpredict -> ESNtrain -> ESNpredict -> ESNtrain -> ...
# to see how predictions change over time, maybe there would be a "core" shape that can be
# like a waveform decomposition of the number, 
